In [1]:
!nvidia-smi

Sat Apr  6 15:45:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch
import torch.nn as nn  # 신경망 모듈, 구축 및 훈련에 사용
from torch.utils.data.dataset import Dataset # 사용자 정의 데이터 셋을 만들어 딥러닝을 할 수 있게 함
import torchvision
import torchvision.transforms as transforms # 이미지 전처리에 필요한 transform 사용
from torch.utils.data import DataLoader # 데이터 관리에 필요한 dataloader
import torch.nn.functional as F # loss or cost gradient와 같은 여러 함수를 사용할 수 있음
from sklearn.metrics import f1_score, confusion_matrix # f1 score와 행렬을 계산 -> 모델의 성능 평가
import numpy as np # numpy 사용
import matplotlib.pyplot as plt #시각화를 위한 차트 사용
from torch.cuda import is_available
from torchvision.datasets import CIFAR10

In [26]:
device='cuda' if is_available() else 'cpu'
lr = 1e-4
batch_size=32
epoch = 10

In [27]:
all_data=CIFAR10(root='.',train=True,transform=transforms.ToTensor(),download=True)

meanRGB=[np.mean(x.numpy(),axis=(1,2)) for x,_ in all_data]
stdRGB=[np.std(x.numpy(),axis=(1,2)) for x,_ in all_data]

meanR = np.mean([m[0] for m in meanRGB])
meanG = np.mean([m[1] for m in meanRGB])
meanB = np.mean([m[2] for m in meanRGB])

stdR = np.mean([s[0] for s in stdRGB])
stdG = np.mean([s[1] for s in stdRGB])
stdB = np.mean([s[2] for s in stdRGB])

print('Mean:',meanR, meanG, meanB)
print('Std:',stdR, stdG, stdB)
mean = [meanR, meanG, meanB]
std = [stdR, stdG, stdB]

Files already downloaded and verified
Mean: 0.49139965 0.48215845 0.4465309
Std: 0.20220213 0.19931543 0.20086348


In [35]:
transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop(227),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(227, scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    # transforms.RandomGrayscale(p=0.2),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),                        # 텐서로 변환
    transforms.Normalize(mean, std)])

In [36]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)

test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [37]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, 96, kernel_size=11, stride=4), # conv1
            torch.nn.ReLU(inplace=True),
            torch.nn.LocalResponseNorm(2),
            torch.nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2), # conv2
            torch.nn.ReLU(inplace=True),
            torch.nn.LocalResponseNorm(2),
            torch.nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1), # conv3
            torch.nn.ReLU(inplace=True)
        )

        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1), # conv4
            torch.nn.ReLU(inplace=True)
        )

        self.layer5 = torch.nn.Sequential(
            torch.nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1), # conv5
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.avgpool = torch.nn.AdaptiveAvgPool2d((6,6))

        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(6*6*256, 4096), # fc1
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(4096, 4096), # fc2
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(4096, num_classes) # fc3
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [38]:
model = Net(in_channels = 3, num_classes=10).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr)
criterion = nn.CrossEntropyLoss()

In [39]:
from tqdm import tqdm

def train(model, train_loader, optimizer, Epoch):
    model.train()
    losses = []
    correct = 0
    total_samples = len(train_loader.dataset)

    with tqdm(total=len(train_loader), unit="batch") as tepoch:
        tepoch.set_description(f"Epoch {Epoch+1}")
        for batch_idx, (image, label) in enumerate(train_loader):
            image = image.to(device)
            label = label.to(device)
            optimizer.zero_grad()
            output = model(image)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
            tepoch.set_postfix(loss=f"{loss.item():.4f}")
            tepoch.update(1)

    train_loss = np.mean(losses)
    accuracy = 100. * correct / total_samples

    return train_loss, accuracy


In [40]:
def evaluate(model, test_loader):
    model.eval()
    losses = []
    correct = 0
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(device)
            label = label.to(device)
            output = model(image)
            loss = criterion(output, label)
            losses.append(loss.item())

            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()

    test_loss = np.mean(losses)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [41]:
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

for Epoch in range(11):
    train_loss, train_accuracy = train(model, trainloader, optimizer, Epoch)
    test_loss, test_accuracy = evaluate(model, testloader)

    # 각 에폭별로 훈련 손실과 정확도, 테스트 손실과 정확도를 저장
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)

    print(f"\n[EPOCH: {Epoch+1}]\tTrain Loss: {train_loss:.4f}\tTrain Accuracy: {train_accuracy} %")
    print(f"[EPOCH: {Epoch+1}]\tTest Loss: {test_loss:.4f}\tTest Accuracy: {test_accuracy} % \n")

Epoch 1: 100%|██████████| 1563/1563 [03:05<00:00,  8.43batch/s, loss=1.2875]



[EPOCH: 1]	Train Loss: 1.5157	Train Accuracy: 43.978 %
[EPOCH: 1]	Test Loss: 1.0930	Test Accuracy: 61.27 % 



Epoch 2: 100%|██████████| 1563/1563 [03:13<00:00,  8.09batch/s, loss=1.4942]



[EPOCH: 2]	Train Loss: 0.9934	Train Accuracy: 65.136 %
[EPOCH: 2]	Test Loss: 0.8307	Test Accuracy: 71.01 % 



Epoch 3: 100%|██████████| 1563/1563 [03:19<00:00,  7.84batch/s, loss=0.4915]


...


[EPOCH: 9]	Train Loss: 0.3486	Train Accuracy: 87.894 %
[EPOCH: 9]	Test Loss: 0.5393	Test Accuracy: 82.35 % 



Epoch 10: 100%|██████████| 1563/1563 [03:25<00:00,  7.61batch/s, loss=1.0098]



[EPOCH: 10]	Train Loss: 0.3084	Train Accuracy: 89.37 %
[EPOCH: 10]	Test Loss: 0.5300	Test Accuracy: 82.82 % 



Epoch 11: 100%|██████████| 1563/1563 [03:24<00:00,  7.64batch/s, loss=0.1719]



[EPOCH: 11]	Train Loss: 0.2750	Train Accuracy: 90.634 %
[EPOCH: 11]	Test Loss: 0.5142	Test Accuracy: 83.84 % 



In [ ]:
plt.figure(figsize=(20, 5))
epochs = range(1, 12)
# 첫 번째 그래프
plt.subplot(1, 2, 1)  # 1행 2열 중 첫 번째 그래프
plt.plot(epochs, train_losses, label='Train Loss',marker='^')
plt.plot(epochs, test_losses, label='Test Loss',marker='^')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# 두 번째 그래프
plt.subplot(1, 2, 2)  # 1행 2열 중 두 번째 그래프
plt.plot(epochs, train_accuracies, label='Train Accuracy',marker='^')
plt.plot(epochs, test_accuracies, label='Test Accuracy',marker='^')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()